In [ ]:
INPUT_PATH = '/kaggle/input/nfl-health-and-safety-helmet-assignment/'
OUTPUT_PATH = '/kaggle/working/'

In [ ]:
import math
import numpy as np

class Point:
    def __init__(self, x, y, is_active=True):
        self.x = x
        self.y = y

        self.is_active = is_active

    def get_distance(self, other=None):
        if other is not None:
            return math.sqrt((self.x - other.x) ** 2 + (self.y - other.y) ** 2)
        else:
            return math.sqrt(self.x ** 2 + self.y ** 2)

    def get_angle(self, other=None):
        if other is not None:
            return math.atan2(self.y - other.y, self.x - other.x) % (2 * math.pi)
        else:
            return math.atan2(self.y, self.x) % (2 * math.pi)

    def get_center(self, other):
        return Point(self.x + (other.x - self.x) / 2.0,
                     self.y + (other.y - self.y) / 2.0)

    def to_polar(self):
        r = self.get_distance()
        theta = self.get_angle()
        return PolarPoint(r, theta)

    def to_numpy(self):
        data_T = np.array([self.x, self.y]).T
        return data_T

    def from_numpy(data):
        x = data[0]
        y = data[1]
        return Point(x, y)

    def is_renderable(self, width=1280, height=720):
        return 0 <= self.x < width and 0 <= self.y < height

    def __getitem__(self, index):
        if isinstance(index, str):
            if index == 'x':
                return self.x
            elif index == 'y':
                return self.y
            else:
                raise ValueError('The index must be either x or y.')
        else:
            raise ValueError('Only a string index is accepted (x, y).')

    def __add__(self, other):
        if isinstance(other, Point):
            return Point(self.x + other.x, self.y + other.y)
        else:
            raise ValueError('Point value is required.')

    def __sub__(self, other):
        if isinstance(other, Point):
            return Point(self.x - other.x, self.y - other.y)
        else:
            raise ValueError('Point value is required.')

    def __mul__(self, other):
        if isinstance(other, Point):
            return Point(self.x * other.x, self.y * other.y)
        else:
            raise ValueError('Point value is required.')

    def __truediv__(self, other):
        if isinstance(other, Point):
            return Point((self.x + 1e-9) / (other.x + 1e-9), (self.y + 1e-9) / (other.y + 1e-9))
        else:
            raise ValueError('Point value is required.')

    def __abs__(self):
        return Point(abs(self.x), abs(self.y))

    def __matmul__(self, A):
        data_T = self.to_numpy()
        data_tr = A @ data_T
        return Point.from_numpy(data_tr)

    def __repr__(self):
        return f'{self.x:.1f} {self.y:.1f}'

class PolarPoint:
    def __init__(self, r, theta):
        self.r = r
        self.theta = theta

    def to_cartesian(self):
        x = self.r * math.cos(self.theta)
        y = self.r * math.sin(self.theta)
        return Point(x, y)

    def __repr__(self):
        return f'{self.r:.1f} {math.degrees(self.theta):.1f}'


In [ ]:
class BoundingBox:
    def __init__(self, left, width, top, height, label='NONE', conf=1.0):
        self.lt = Point(left, top)
        self.rb = Point(left + width, top + height)
        self.label = label
        self.conf = conf

    def get_width(self):
        return self.rb.x - self.lt.x

    def get_height(self):
        return self.rb.y - self.lt.y

    def get_iou(self, other):
        x1, y1 = self.lt.x, self.lt.y
        x2, y2 = self.rb.x, self.rb.y

        x3, y3 = other.lt.x, other.lt.y
        x4, y4 = other.rb.x, other.rb.y

        if x1 < x2 and y1 < y2 and x3 < x4 and y3 < y4:
            x_left = max(x1, x3)
            y_top = max(y1, y3)
            x_right = min(x2, x4)
            y_bottom = min(y2, y4)

            if x_right < x_left or y_bottom < y_top:
                return 0.0

            intersection_area = (x_right - x_left) * (y_bottom - y_top)

            bb1_area = (x2 - x1) * (y2 - y1)
            bb2_area = (x4 - x3) * (y4 - y3)

            iou = intersection_area / float(bb1_area + bb2_area - intersection_area)

            if 0.0 <= iou <= 1.0:
                return iou
            else:
                return 0.0
        else:
            return 0.0


In [ ]:
import math

class Player:
    def __init__(self, name, frame_id, x, y, s, a, dis, o, direction, event):
        self.name = name
        self.frame_id = frame_id
        self.x = x
        self.y = y
        self.speed = s
        self.acceleration = a
        self.distance = dis
        self.orientation = math.radians(o)
        self.direction = math.radians(direction)
        self.event = event

        self.is_active = True


In [ ]:
import math

class Angle(float):
    __max_degree = 2 * math.pi

    def __new__(cls, value, *args, **kwargs):
        return super(cls, cls).__new__(cls, value)

    def is_in(self, a, b):
        x, a, b = float(self), float(a), float(b)

        if a > b:
            if x > b:
                a -= 2 * math.pi
                x -= 2 * math.pi
            else:
                b += 2 * math.pi
                x += 2 * math.pi

        return a <= x <= b

    def __add__(self, other):
        res = super(Angle, self).__add__(other)
        return self.__class__(self.__apply_bounds(res))

    def __sub__(self, other):
        res = super(Angle, self).__sub__(other)

        if res < -math.pi:
            res += 2 * math.pi

        elif res > math.pi:
            res -= 2 * math.pi

        return self.__class__(res)

    def __mul__(self, other):
        res = super(Angle, self).__mul__(other)
        return self.__class__(self.__apply_bounds(res))

    def __div__(self, other):
        res = super(Angle, self).__div__(other)
        return self.__class__(res)

    def __repr__(self):
        return f'{math.degrees(self):.1f}'

    def __str__(self):
        return f'{math.degrees(self):.1f}'

    def __apply_bounds(self, x):
        return x % self.__max_degree


In [ ]:
import math

class Point3D:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def get_distance(self, other):
        return math.sqrt((self.x - other.x) ** 2 + (self.y - other.y) ** 2 + (self.z - other.z) ** 2)

    def get_angle(self, rotation):
        alpha = -math.atan2(self.y, math.sqrt(self.x ** 2 + self.z ** 2)) + rotation.x
        beta = -math.atan2(self.z, self.x) + math.pi / 2.0 + rotation.y
        gamma = rotation.z
        return Angle3D(alpha, beta, gamma)

    def get_center(self, other):
        return Point3D(self.x + (other.x - self.x) / 2.0,
                          self.y + (other.y - self.y) / 2.0,
                          self.z + (other.z - self.z) / 2.0)

    def to_spherical(self):
        r = math.sqrt(self.x ** 2 + self.y ** 2 + self.z ** 2)
        theta = math.atan2(math.sqrt(self.x ** 2 + self.y ** 2), self.z)
        fi = math.atan(self.y / (self.x + 1e-9))

        if self.x < 0:
            fi += math.pi

        return SphericalPoint(r, theta, fi)

    def __add__(self, other):
        if isinstance(other, Point3D):
            return Point3D(self.x + other.x, self.y + other.y, self.z + other.z)
        else:
            raise ValueError('Point3D value is required.')

    def __sub__(self, other):
        if isinstance(other, Point3D):
            return Point3D(self.x - other.x, self.y - other.y, self.z - other.z)
        else:
            raise ValueError('Point3D value is required.')

    def __mul__(self, value):
        if isinstance(value, int) or isinstance(value, float):
            return Point3D(self.x * value, self.y * value, self.z * value)
        else:
            raise ValueError('Integer or float value is required.')

    def __truediv__(self, value):
        if isinstance(value, int) or isinstance(value, float):
            return Point3D(self.x / value, self.y / value, self.z / value)
        else:
            raise ValueError('Integer or float value is required.')

    def __eq__(self, other):
        is_x_equal = math.isclose(self.x, other.x)
        is_y_equal = math.isclose(self.y, other.y)
        is_z_equal = math.isclose(self.z, other.z)
        return is_x_equal and is_y_equal and is_z_equal

    def __repr__(self):
        return f'{self.x:.1f} {self.y:.1f} {self.z:.1f}'

class Angle3D(Point3D):
    def __init__(self, x, y, z):
        super(Angle3D, self).__init__(x, y, z)
        self.x = Angle(self.x)
        self.y = Angle(self.y)
        self.z = Angle(self.z)

    def __add__(self, other):
        if isinstance(other, Point3D) or isinstance(other, Angle3D) or isinstance(other, AngleShift3D):
            return Angle3D(self.x + other.x, self.y + other.y, self.z + other.z)
        else:
            raise ValueError('Point3D, Angle3D or AngleShift3D value is required.')

    def __repr__(self):
        return f'{self.x} {self.y} {self.z}'

class AngleShift3D(Point3D):
    def __init__(self, x, y, z):
        super(AngleShift3D, self).__init__(x, y, z)

    def __repr__(self):
        return f'{math.degrees(self.x):.1f} {math.degrees(self.y):.1f} {math.degrees(self.z):.1f}'

class SphericalPoint:
    def __init__(self, r, theta, fi):
        self.r = r
        self.theta = Angle(theta % (2 * math.pi))
        self.fi = Angle(fi % (2 * math.pi))

    def to_cartesian(self):
        x = self.r * math.cos(self.fi) * math.sin(self.theta)
        y = self.r * math.sin(self.fi) * math.sin(self.theta)
        z = self.r * math.cos(self.theta)
        return Point3D(x, y, z)

    def __repr__(self):
        return f'{self.r:.1f} {self.theta} {self.fi}'


In [ ]:
import math
import numpy as np

def print_if(condition, msg='', end='\n'):
    if condition:
        print(msg, end=end)

def with_plus(x):
    if type(x) is not float:
        x = math.degrees(x)

    return f' {x:.1f}' if math.isclose(x, 0.0) else f'{x:+.1f}'

def get_dicts_merged(dicts, dict_keys):
    dict_merged = {}

    for dict_now, dict_key in zip(dicts, dict_keys):
        for key, value in dict_now.items():
            dict_merged[f'{dict_key}_{key}'] = value

    return dict_merged

def match_bboxes(bboxes1, bboxes2, iou_thresh):
    matches = []

    for i, bbox1 in enumerate(bboxes1):
        for j, bbox2 in enumerate(bboxes2):
            iou = bbox1.get_iou(bbox2)

            if iou > iou_thresh:
                matches.append(tuple((i, j, iou)))

    matches.sort(key=lambda x: x[2], reverse=True)
    match_dict = {}

    for first, second, iou in matches:
        if first not in match_dict.keys() and second not in match_dict.values():
            match_dict[first] = second

    return match_dict

def get_2d_from_3d(c_x, c_y, c_z, theta_x, theta_y, theta_z, e_x, e_y, e_z, a_x, a_y, a_z):
    cos_x = math.cos(theta_x)
    cos_y = math.cos(theta_y)
    cos_z = math.cos(theta_z)

    sin_x = math.sin(theta_x)
    sin_y = math.sin(theta_y)
    sin_z = math.sin(theta_z)

    arg1 = np.array([[1, 0, 0], [0, cos_x, sin_x], [0, -sin_x, cos_x]])
    arg2 = np.array([[cos_y, 0, -sin_y], [0, 1, 0], [sin_y, 0, cos_y]])
    arg3 = np.array([[cos_z, sin_z, 0], [-sin_z, cos_z, 0], [0, 0, 1]])
    arg4 = np.array([a_x - c_x, a_y - c_y, a_z - c_z])
    d_x, d_y, d_z = arg1 @ arg2 @ arg3 @ arg4

    b_x = (d_x * e_z) / (d_z + 1e-9) + e_x
    b_y = (d_y * e_z) / (d_z + 1e-9) + e_y
    return b_x, b_y

def get_fibonacci_sphere(n, r):
    points = []
    phi = math.pi * (3.0 - math.sqrt(5.0))

    for i in range(n):
        y = 1 - (i / float(n - 1)) * 2
        radius = math.sqrt(1 - y * y)
        theta = phi * i

        x = math.cos(theta) * radius
        z = math.sin(theta) * radius
        points.append(tuple((x, y, z)))

    return points

def get_value_min_max(value, eps):
    return value - eps, value + eps


In [ ]:
import os
import cv2
import tqdm
import pandas as pd

def read_labels(filename, videonames):
    df = pd.read_csv(filename)
    bboxes_by_sample = {}

    for videoname in videonames:
        df_now = df[df['video_frame'].str.contains(videoname)]
        bboxes = {}

        for i, row in df_now.iterrows():
            index = row['frame']

            if index not in bboxes.keys():
                bboxes[index] = []

            bbox = BoundingBox(row['left'], row['width'], row['top'], row['height'], label=row['label'])
            bboxes[index].append(bbox)

        bboxes_by_sample[videoname] = bboxes

    return df_now, bboxes_by_sample

def read_baseline(filename, videonames, confidence):
    df = pd.read_csv(filename)
    bboxes_by_sample = {}

    for videoname in videonames:
        df_now = df[df['video_frame'].str.contains(videoname)]
        df_now = df_now[df_now['conf'] >= confidence]
        bboxes = {}

        for i, row in df_now.iterrows():
            index = int(row['video_frame'].split('_')[-1])

            if index not in bboxes.keys():
                bboxes[index] = []

            bbox = BoundingBox(row['left'], row['width'], row['top'], row['height'], conf=row['conf'])
            bboxes[index].append(bbox)

        bboxes_by_sample[videoname] = bboxes

    return df_now, bboxes_by_sample

def read_tracking_data(filename, videonames):
    df = pd.read_csv(filename)
    players_by_sample = {}

    for videoname in videonames:
        videoname_parts = videoname.split('_')
        gameKey, playID = int(videoname_parts[0]), int(videoname_parts[1])
        df_now = pd.DataFrame(df[(df['gameKey'] == gameKey) & (df['playID'] == playID)])
        df_now['time'] = pd.to_datetime(df_now['time'])
        df_now['event'] = df_now['event'].fillna('no')
        time_min = df_now[df_now['event'].str.contains('ball_snap')]['time'].min()
        players = {}

        for player_name in df_now['player'].unique().tolist():
            for i, row in df_now[df_now['player'] == player_name].iterrows():
                frame_id = round(pd.Timedelta(row['time'] - time_min).total_seconds() * 10) * 6 + 10

                if frame_id < 0:
                    continue

                if frame_id not in players.keys():
                    players[frame_id] = []

                x = row['x']
                y = row['y']
                s = row['s']
                a = row['a']
                dis = row['dis']
                o = row['o']
                direction = row['dir']
                event = row['event']
                player = Player(player_name, frame_id, x, y, s, a, dis, o, direction, event)
                players[frame_id].append(player)

        player_frame_ids = list(players.keys())

        for frame_id in player_frame_ids:
            if len(players[frame_id]) < 22:
                players.pop(frame_id, None)

        players_by_sample[videoname] = players

    return players_by_sample

def read_sample_submission(filename):
    df = pd.read_csv(filename)
    df.drop(df.index, inplace=True)
    return df

def read_submission(filename, videoname):
    df = pd.read_csv(filename)
    df = df[df['video_frame'].str.contains(videoname)]
    bboxes = {}

    for i, row in df.iterrows():
        frame_id = int(row['video_frame'].split('_')[-1])

        if frame_id not in bboxes.keys():
            bboxes[frame_id] = []

        bbox = BoundingBox(row['left'], row['width'], row['top'], row['height'], label=row['label'])
        bboxes[frame_id].append(bbox)

    return bboxes

def read_image_labels(filename):
    df = pd.read_csv(filename)
    bboxes = {}

    with tqdm.tqdm(total=len(df)) as tqdm_bar:
        for i, row in df.iterrows():
            image_name = row['image']

            if image_name not in bboxes.keys():
                bboxes[image_name] = []

            bbox = BoundingBox(row['left'], row['width'], row['top'], row['height'], label=row['label'])
            bboxes[image_name].append(bbox)
            tqdm_bar.update(1)

    return bboxes

def write_submission(filename, df):
    df.to_csv(filename, index=False)

def get_ball_snap_frame_id(player_dict):
    min_frame_id = min([k for k, v in player_dict.items() if any(p.event == 'ball_snap' for p in v)])
    return min_frame_id

def read_specific_frame(video_partname, frame_id):
    video_name = INPUT_PATH + f'test/{video_partname}.mp4'

    if not os.path.exists(video_name):
        print(f'{video_name} not found.')
        return None

    cap = cv2.VideoCapture(video_name)

    if not cap.isOpened():
        print(f'Can\'t open the video.')
        return None

    for i in range(frame_id):
        ret, frame = cap.read()

        if not ret:
            print(f'Can\'t read the frame.')
            return None

    cap.release()
    return frame


In [ ]:
import cv2

def draw_points(frame, points, thickness, color, cluster_only):
    height, width, _ = frame.shape

    for i, point in enumerate(points):
        if not cluster_only or i in points.cluster:
            if point.is_renderable(width, height):
                cv2.circle(frame, (int(point.x), int(point.y)), thickness, color, -1)

def draw_lines(frame, points1, points2, matches, cluster_only):
    height, width, _ = frame.shape

    for match in matches:
        first, second = match.first, match.second
        p1 = points1[first]
        p2 = points2[second]

        if not cluster_only or first in points1.cluster and second in points2.cluster:
            if p1.is_renderable(width, height) and p2.is_renderable(width, height):
                cv2.line(frame, (int(p1.x), int(p1.y)), (int(p2.x), int(p2.y)), (0, 255, 0), thickness=1)

def draw_point_matches(frame_initial, projected_points, detected_points, matches, cluster_only=False):
    frame = frame_initial.copy()
    draw_points(frame, projected_points, 5, (255, 0, 0), cluster_only)
    draw_points(frame, detected_points, 3, (0, 255, 0), cluster_only)
    draw_lines(frame, projected_points, detected_points, matches, cluster_only)
    return frame

def draw_points_only(frame_initial, points, thickness, color, cluster_only=False):
    frame = frame_initial.copy()
    draw_points(frame, points, thickness, color, cluster_only)
    return frame


In [ ]:
import math

class SphericalPositions:
    __max_const_theta = math.pi
    __max_const_fi = 2 * math.pi

    def __init__(self, n, r, angle_eps=7.5, r_num=5, r_eps=1.0):
        self.__n = n
        self.__r = r

        self.__angle_eps_init = angle_eps
        self.__r_eps_init = r_eps

        self.__angle_eps = self.__angle_eps_init
        self.__r_eps = self.__r_eps_init

        self.__r_num = r_num

        self.__guesses = []
        self.__rs = []

    def update_r(self, r):
        self.__r = r

    def init(self):
        min_theta = 0.0
        max_theta = math.pi / 2.0

        min_fi = 0.0
        max_fi = 2.0 * math.pi

        self.__angle_eps = self.__angle_eps_init
        self.__r_eps = self.__r_eps_init

        self.__generate(min_theta, max_theta, min_fi, max_fi, r_num=1, r_eps=1.0)

    def update(self, theta, fi):
        min_theta, max_theta = get_value_min_max(math.degrees(theta), self.__angle_eps)
        min_fi, max_fi = get_value_min_max(math.degrees(fi), self.__angle_eps)

        min_theta = math.radians(min_theta)
        max_theta = math.radians(max_theta)

        min_fi = math.radians(min_fi)
        max_fi = math.radians(max_fi)

        self.__generate(min_theta, max_theta, min_fi, max_fi, self.__r_num, self.__r_eps)

    def decrease(self):
        self.__angle_eps = self.__angle_eps / 2.0
        self.__r_eps = self.__r_eps / 2.0

    def __generate(self, min_theta, max_theta, min_fi, max_fi, r_num, r_eps):
        n = self.__n

        self.__guesses.clear()
        self.__rs.clear()
        self.__rs.append(self.__r)

        if r_num > 1 and r_num % 2 == 1:
            r_eps_num = (r_num - 1) // 2
            n = n // r_num

            for i in range(r_eps_num):
                self.__rs.insert(0, self.__r - r_eps * (i + 1))
                self.__rs.append(self.__r + r_eps * (i + 1))

        sphere_share = ((max_theta - min_theta) / self.__max_const_theta) * ((max_fi - min_fi) / self.__max_const_fi)
        n = int(n / sphere_share)

        points = [Point3D(x[0], x[1], x[2]) for x in get_fibonacci_sphere(n, self.__r)]
        points = [x.to_spherical() for x in points]
        points = [x for x in points if (min_theta < x.theta < max_theta) and (min_fi < x.fi < max_fi)]

        for point in points:
            for r in self.__rs:
                guess_now = SphericalPoint(r, point.theta, point.fi)
                self.__guesses.append(guess_now)

    def __getitem__(self, index):
        if isinstance(index, int):
            return self.__guesses[index]
        else:
            raise ValueError('Only an integer index is accepted.')

    def __len__(self):
        return len(self.__guesses)


In [ ]:
import math
import numpy as np
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

class Match:
    def __init__(self, first, second, distance):
        self.first = first
        self.second = second
        self.distance = distance

    def __repr__(self):
        return f'{self.first} {self.second} {self.distance:.3f}'

class PointCloud:
    def __init__(self, points, cluster=None):
        self.points = points
        self.cluster = cluster if cluster is not None else list(range(len(self)))

    def with_cluster(self, cluster):
        return PointCloud(self.points, cluster)

    def __getitem__(self, index):
        if isinstance(index, slice):
            return PointCloud(self.points[index.start: index.stop: index.step])

        elif isinstance(index, list):
            return PointCloud([self[i] for i in index])

        else:
            if 0 <= index < len(self.points):
                return self.points[index]
            else:
                raise ValueError('The element is not accessible by this index.')

    def __iter__(self):
        return self.points.__iter__()

    def __len__(self):
        return len(self.points)

    def get_match_distance(self, cost_matrix, min_length):
        _, assigned_indices = linear_sum_assignment(cost_matrix)
        matches = [Match(i, j, cost_matrix[i][j]) for i, j in enumerate(assigned_indices)]
        distance = np.sum([m.distance for m in matches]) / min_length
        return matches, distance

    def get_alive_indices(self):
        return [i for i, point in enumerate(self) if point.is_active and i in self.cluster]

    def get_matches_inverted(self, matches, self_indices, other_indices):
        matches_updated = []

        for match in matches:
            first_inverted = self_indices[match.first]
            second_inverted = other_indices[match.second]
            matches_updated.append(Match(first_inverted, second_inverted, match.distance))

        return matches_updated

    def get_distance(self, other):
        min_length = min(len(self), len(other)) + 1e-9

        self_indices = self.get_alive_indices()
        other_indices = other.get_alive_indices()

        self_points = self[self_indices].to_numpy()
        other_points = other[other_indices].to_numpy()

        cost_matrix = cdist(self_points, other_points)

        matches, distance = self.get_match_distance(cost_matrix, min_length)
        matches_inv, distance_inv = self.get_match_distance(cost_matrix.T, min_length)

        matches = self.get_matches_inverted(matches, self_indices, other_indices)
        matches_inv = self.get_matches_inverted(matches_inv, other_indices, self_indices)

        if distance_inv < distance:
            matches = [Match(m.second, m.first, m.distance) for m in matches_inv]
            distance = distance_inv

        self_indices_used = [m.first for m in matches]
        other_indices_used = [m.second for m in matches]
        return distance, matches

    def to_numpy(self):
        return np.array([[point.x, point.y] for point in self.points])

    def get_center(self):
        median_x = np.median([p.x for p in self])
        median_y = np.median([p.y for p in self])
        return Point(median_x, median_y)

    def get_min_max(self):
        x = [p.x for p in self]
        y = [p.y for p in self]

        min_x, min_y = np.min(x), np.min(y)
        max_x, max_y = np.max(x), np.max(y)

        min_point = Point(min_x, min_y)
        max_point = Point(max_x, max_y)
        return min_point, max_point

    def get_point_cloud_cluster(self, index_dict, eps=1e-3):
        cluster_first = []

        max_index_value = max([abs(x + 1 if x < 0 else x) for x in index_dict.values()])
        max_index_key = [k for k, v in index_dict.items() if abs(v + 1 if v < 0 else v) == max_index_value][0]

        first, second = 'x', 'y'
        first_keys, second_keys = ['left', 'right'], ['top', 'bottom']

        if max_index_key not in ['left', 'right']:
            first, second = second, first
            first_keys, second_keys = second_keys, first_keys

        min_dict, max_dict = {}, {}
        sorted_by_first = sorted(self, key=lambda x: x[first])

        min_dict[first] = sorted_by_first[index_dict[first_keys[0]]][first]
        max_dict[first] = sorted_by_first[index_dict[first_keys[1]]][first]

        for i, p in enumerate(self):
            if min_dict[first] - eps < p[first] < max_dict[first] + eps:
                cluster_first.append(i)

        sorted_by_second = sorted([p for i, p in enumerate(self) if i in cluster_first], key=lambda x: x[second])

        min_dict[second] = sorted_by_second[index_dict[second_keys[0]]][second]
        max_dict[second] = sorted_by_second[index_dict[second_keys[1]]][second]

        self.cluster = []

        for i, p in enumerate(self):
            if i in cluster_first:
                if min_dict[second] - eps < p[second] < max_dict[second] + eps:
                    self.cluster.append(i)

                elif math.isclose(p[first], min_dict[first]) or math.isclose(p[first], max_dict[first]):
                    self.cluster.append(i)

        return self.cluster

    def get_centered(self, center):
        points_updated = []

        for point in self.points:
            point_updated = point - center
            points_updated.append(point_updated)

        return PointCloud(points_updated, self.cluster)

    def get_moved(self, other):
        self_cluster = self[self.cluster]
        other_cluster = other[other.cluster]

        self_center = self_cluster.get_center()
        other_center = other_cluster.get_center()

        diff_point = other_center - self_center
        points_updated = []

        for point in self.points:
            point_updated = point + diff_point
            points_updated.append(point_updated)

        return PointCloud(points_updated, self.cluster)

    def get_scaled(self, other):
        self_cluster = self[self.cluster]
        other_cluster = other[other.cluster]

        self_center = self_cluster.get_center()

        self_centered = self_cluster.get_centered(self_center)
        other_centered = other_cluster.get_centered(self_center)

        self_min, self_max = self_centered.get_min_max()
        other_min, other_max = other_centered.get_min_max()

        scale_min = other_min / self_min
        scale_max = other_max / self_max

        points_updated = []

        for point in self.points:
            diff_point = point - self_center

            if diff_point.x < 0.0 and diff_point.y < 0.0:
                scale_point = Point(scale_min.x, scale_min.y)

            elif diff_point.x < 0.0 and diff_point.y > 0.0:
                scale_point = Point(scale_min.x, scale_max.y)

            elif diff_point.x > 0.0 and diff_point.y < 0.0:
                scale_point = Point(scale_max.x, scale_min.y)

            else:
                scale_point = Point(scale_max.x, scale_max.y)

            point_updated = self_center + diff_point * scale_point
            points_updated.append(point_updated)

        return PointCloud(points_updated, self.cluster)


In [ ]:
import itertools
from copy import deepcopy

class PointClustering:
    n = 4

    def __init__(self, projection_num, detection_num, distance=1e+6):
        self.projection_num = projection_num
        self.detection_num = detection_num

        self.__i = 0
        self.__combinations = []
        self.__best_distance = distance

    def generate(self, min_outlier_num=-1, max_outlier_num=2):
        difference = self.detection_num - self.projection_num
        detections_more = True

        if difference < 0:
            difference = -difference
            detections_more = False

        combinations = []

        for m in range(difference - max_outlier_num, difference + max_outlier_num + 1):
            if min_outlier_num == -1 or not difference - min_outlier_num <= m <= min_outlier_num:
                res = list(x for x in itertools.product(range(0, m + 1), repeat=self.n) if sum(x) == m)
                combinations.extend(res)

        combinations = sorted(combinations, key=lambda x: (max(x) / sum(x) if sum(x) != 0 else 1, sum(x)), reverse=detections_more)

        self.__combinations = []

        for left, top, right, bottom in combinations:
            index_dict = {}
            index_dict['projections'] = {'left': 0, 'top': 0, 'right': -1, 'bottom': -1}
            index_dict['detections'] = {'left': 0, 'top': 0, 'right': -1, 'bottom': -1}

            if not detections_more:
                index_dict['projections'] = {'left': left, 'top': top, 'right': -(right + 1), 'bottom': -(bottom + 1)}
            else:
                index_dict['detections'] = {'left': left, 'top': top, 'right': -(right + 1), 'bottom': -(bottom + 1)}

            self.__combinations.append(index_dict)

        return self.__combinations

    def update_if_less(self, index_dict, distance):
        if distance < self.__best_distance:
            self.__best_distance = distance
            return True
        else:
            return False

    def __iter__(self):
        self.__i = -1
        return self

    def __next__(self):
        if self.__i < len(self.__combinations) - 1:
            self.__i += 1
            return self.__combinations[self.__i]
        else:
            raise StopIteration


In [ ]:
import cv2
import math
import tqdm
import itertools
import numpy as np

class Camera:
    def __init__(self, c_x=0.0, c_y=0.0, c_z=0.0, t_x=0.0, t_y=0.0, t_z=0.0, e_x=0.5, e_y=0.5, e_z=1.0):
        self.c = Point3D(c_x, c_y, c_z)
        self.theta = Angle3D(t_x, t_y, t_z)
        self.e = Point3D(e_x, e_y, e_z)

    def copy_from(self, other):
        self.c = Point3D(other.c.x, other.c.y, other.c.z)
        self.theta = Angle3D(other.theta.x, other.theta.y, other.theta.z)
        self.e = Point3D(other.e.x, other.e.y, other.e.z)

    def get_2d_from_3d(self, a_x, a_y, a_z):
        return get_2d_from_3d(self.c.x, self.c.y, self.c.z,
                              self.theta.x, self.theta.y, self.theta.z,
                              self.e.x, self.e.y, self.e.z,
                              a_x, a_y, a_z)

    def get_line_info(self, other):
        c_delta = self.c - other.c

        s = f'{with_plus(c_delta.x)} {with_plus(c_delta.y)} {with_plus(c_delta.z)} '
        s += f'{with_plus(self.theta.x - other.theta.x)} '
        s += f'{with_plus(self.theta.y - other.theta.y)} '
        s += f'{with_plus(self.theta.z - other.theta.z)}\t'

        s += f'{self.c.x:.1f} {self.c.y:.1f} {self.c.z:.1f} '
        s += f'{self.theta.x} '
        s += f'{self.theta.y} '
        s += f'{self.theta.z}'
        return s

    def get_full_info(self, other):
        s = ''

        c_delta = self.c - other.c

        s += f'\tc_x: {self.c.x:.1f} ({c_delta.x:+.1f})\n'
        s += f'\tc_y: {self.c.y:.1f} ({c_delta.y:+.1f})\n'
        s += f'\tc_z: {self.c.z:.1f} ({c_delta.z:+.1f})\n\n'

        s += f'\ttheta_x: {self.theta.x} ({self.theta.x - other.theta.x})\n'
        s += f'\ttheta_y: {self.theta.y} ({self.theta.y - other.theta.y})\n'
        s += f'\ttheta_z: {self.theta.z} ({self.theta.z - other.theta.z})\n\n'

        s += f'\te_x: {self.e.x:.1f}\n'
        s += f'\te_y: {self.e.y:.1f}\n'
        s += f'\te_z: {self.e.z:.1f}'
        return s

    def __repr__(self):
        s = ''

        s += f'\tc_x: {self.c.x:.1f}\n'
        s += f'\tc_y: {self.c.y:.1f}\n'
        s += f'\tc_z: {self.c.z:.1f}\n\n'

        s += f'\ttheta_x: {self.theta.x}\n'
        s += f'\ttheta_y: {self.theta.y}\n'
        s += f'\ttheta_z: {self.theta.z}\n\n'

        s += f'\te_x: {self.e.x:.1f}\n'
        s += f'\te_y: {self.e.y:.1f}\n'
        s += f'\te_z: {self.e.z:.1f}\n\n'
        return s

class BestResult:
    def __init__(self):
        self.distance = 1e+6
        self.camera = Camera()
        self.points = []
        self.matches = []
        self.index = -1

    def copy_from(self, other):
        self.distance = other.distance
        self.camera.copy_from(other.camera)
        self.points = other.points[:]
        self.matches = other.matches[:]
        self.index = -1

    def update_results(self, points, distance, matches):
        self.points = points[:]
        self.distance = distance
        self.matches = matches[:]

    def update_if_less(self, distance, camera, points, matches, index=-1):
        if distance < self.distance:
            self.distance = distance
            self.camera.copy_from(camera)
            self.points = points[:]
            self.matches = matches[:]
            self.index = index
            return True
        else:
            return False

class CameraRotations:
    def __init__(self, is_endzone=False):
        if not is_endzone:
            self.x = np.linspace(0.0, math.pi, 2)
            self.y = np.linspace(0.0, math.pi, 2)
            self.z = np.linspace(-3 * math.pi / 18.0, 3 * math.pi / 18.0, 7)
        else:
            self.x = np.linspace(0.0, math.pi, 2)
            self.y = np.linspace(0.0, math.pi, 2)
            self.z = np.linspace(0.0, 2 * math.pi, 4, endpoint=False)

        self.guesses = [Angle3D(x[0], x[1], x[2]) for x in itertools.product(self.x, self.y, self.z)]

class DirectionList:
    def __init__(self, params):
        self.__c_init = [0.0]
        self.__t_init = [0.0]

        self.__position_shift = params['position_shift']
        self.__rotation_shift = math.radians(params['rotation_shift'])
        self.__shift_num = params['shift_num']
        self.__multiply_coeff = params['multiply_coeff']
        self.__index = params['initial_index']

        if self.__position_shift > 0.0:
            for i in range(1, self.__shift_num + 1):
                self.__c_init.insert(0, -i * self.__position_shift)
                self.__c_init.append(i * self.__position_shift)

        if self.__rotation_shift > 0.0:
            for i in range(1, self.__shift_num + 1):
                self.__t_init.insert(0, -i * self.__rotation_shift)
                self.__t_init.append(i * self.__rotation_shift)

        z_angles = [0.0] if len(self.__c_init) == 1 else self.__t_init

        self.__directions = [x for x in itertools.product(self.__c_init, self.__c_init, self.__c_init,
                                                          self.__t_init, self.__t_init, z_angles)]

        self.__directions = [tuple((Point3D(x[0], x[1], x[2]),
                                    AngleShift3D(x[3], x[4], x[5]))) for x in self.__directions]

    def reset(self):
        self.__index = 0

    def increase(self):
        self.__index += 1

    def decrease(self):
        if self.__index > 0:
            self.__index -= 1
            return True
        else:
            return False

    def __getitem__(self, index):
        if isinstance(index, int):
            return [x * (self.__multiply_coeff ** self.__index) for x in self.__directions[index]]
        else:
            raise ValueError('Only an integer index is accepted.')

    def __len__(self):
        return len(self.__directions)

class CameraMapping:
    mean_player_height = 1.75
    init_camera_height = 50.0

    def __init__(self, players, detected_points, frame_initial, is_endzone):
        self.players = players
        self.detected_points = PointCloud(detected_points)

        self.player_cluster = list(range(len(self.players)))
        self.detection_cluster = list(range(len(self.detected_points)))

        self.frame_initial = frame_initial
        self.is_endzone = is_endzone
        self.height, self.width, _ = self.frame_initial.shape

        ball_player = [x for x in players if x.event == 'ball_snap'][0]
        self.game_center = Point3D(ball_player.x, ball_player.y, self.mean_player_height)
        self.camera = Camera(self.game_center.x, self.game_center.y, self.init_camera_height)
        self.rotation = Angle3D(0.0, 0.0, 0.0)

        self.camera_initial = Camera()
        self.camera_previous = Camera()

        self.best = BestResult()
        self.positions = SphericalPositions(n=125, r=1.0)
        self.rotations = CameraRotations(self.is_endzone)
        self.direction_list = []

        self.pointClustering = PointClustering(len(self.players), len(self.detected_points))
        self.pointClustering.generate()

    def update_tracking_data(self, frame, players, detected_points):
        self.frame_initial = frame

        self.players = players
        self.detected_points = PointCloud(detected_points)

        self.player_cluster = list(range(len(self.players)))
        self.detection_cluster = list(range(len(self.detected_points)))

    def update_clusters(self, camera, distance, previous_distance, increase_coeff):
        initial_projections = self.get_player_projections(camera)
        self.pointClustering = PointClustering(len(self.players), len(self.detected_points), distance)

        for outlier_num in range(0, 11):
            self.pointClustering.generate(min_outlier_num=outlier_num - 1, max_outlier_num=outlier_num)
            is_found = False

            for index_dict in self.pointClustering:
                initial_projections.get_point_cloud_cluster(index_dict['projections'])
                self.detected_points.get_point_cloud_cluster(index_dict['detections'])

                distance, matches = initial_projections.get_distance(self.detected_points)
                is_updated = self.pointClustering.update_if_less(index_dict, distance)

                if is_updated:
                    self.player_cluster = initial_projections.cluster
                    self.detection_cluster = self.detected_points.cluster

                    if distance < previous_distance * increase_coeff:
                        is_found = True
                        break

            if is_found:
                break

    def get_best_rotations(self, with_height_adjusted=True, is_debug=True):
        best_distance = 1e+6

        for i in tqdm.tqdm(range(len(self.rotations.guesses))):
            theta = self.rotations.guesses[i]

            camera_rotated = Camera()
            camera_rotated.copy_from(self.camera)
            camera_rotated.theta = Angle3D(theta.x, theta.y, theta.z)
            initial_projections = self.get_player_projections(camera_rotated)

            for index_dict in self.pointClustering:
                initial_projections.get_point_cloud_cluster(index_dict['projections'])
                self.detected_points.get_point_cloud_cluster(index_dict['detections'])

                moved_points = initial_projections.get_moved(self.detected_points)
                scaled_points = moved_points.get_scaled(self.detected_points)
                distance, matches = scaled_points.get_distance(self.detected_points)

                if distance < best_distance:
                    self.rotation = Angle3D(theta.x, theta.y, theta.z)
                    self.pointClustering.update_if_less(index_dict, distance)

                    self.player_cluster = initial_projections.cluster
                    self.detection_cluster = self.detected_points.cluster

                    best_distance = distance

                    if is_debug:
                        self.write_point_image('../rotation_imgs/0.0.detected.png', self.detected_points, 3, (0, 255, 0))
                        self.write_point_image('../rotation_imgs/0.1.projected.png', initial_projections, 5, (255, 0, 0))
                        self.write_match_image('../rotation_imgs/0.initial.png', initial_projections, self.detected_points, matches)
                        self.write_match_image('../rotation_imgs/1.moved.png', moved_points, self.detected_points, matches)
                        self.write_match_image('../rotation_imgs/2.scaled.png', scaled_points, self.detected_points, matches)

        if with_height_adjusted:
            r = self.adjust_camera_height()
            self.positions.update_r(r)
            self.camera.c.z = r

        return self.rotation

    def adjust_camera_height(self, eps=0.1):
        def get_point_cloud_distance(height):
            best = BestResult()
            best.camera = Camera(self.camera.c.x, self.camera.c.y, height)
            self.look_at_center(best.camera)
            self.find_best_parameters(best, with_position=False)
            return best.distance

        down_height = 1
        up_height = 101

        left_distance = get_point_cloud_distance(down_height)
        right_distance = get_point_cloud_distance(up_height)

        while True:
            mid_height = down_height + (up_height - down_height) / 2.0
            mid_distance = get_point_cloud_distance(mid_height)

            mid_down_height = mid_height - eps
            mid_left_distance = get_point_cloud_distance(mid_down_height)

            mid_up_height = mid_height + eps
            mid_right_distance = get_point_cloud_distance(mid_up_height)

            if mid_left_distance < mid_distance:
                up_height = mid_height
                right_distance = mid_distance
            else:
                down_height = mid_height
                left_distance = mid_distance

            if up_height - down_height < eps:
                break

        mid_height = down_height + (up_height - down_height) / 2.0
        return mid_height

    def write_point_image(self, filename, points, thickness, color):
        img = np.zeros((self.height, self.width, 3), dtype=np.uint8)
        img = draw_points_only(img, points, thickness, color, cluster_only=True)
        cv2.imwrite(filename, img)

    def write_match_image(self, filename, projected_points, detected_points, matches):
        img = np.zeros((self.height, self.width, 3), dtype=np.uint8)
        img = draw_point_matches(img, projected_points, detected_points, matches, cluster_only=True)
        cv2.imwrite(filename, img)

    def find_best_position(self):
        self.positions.init()

        if not self.is_endzone:
            best_point, best_distance = self.estimate_spherical_positions()
            print(f'\nSideline Initial: {best_point}, {best_distance:.3f}\n')

            for i in range(2):
                self.positions.update(best_point.theta, best_point.fi)
                best_point, best_distance = self.estimate_spherical_positions()
                self.positions.decrease()
                print(f'\nSideline Iteration {i + 1}: {best_point}, {best_distance:.3f}')

                if i < 1:
                    print()
        else:
            best_points, best_distances = self.estimate_spherical_positions(return_two_results=True)
            best_distance_points = {}

            best_points_repr = ' - '.join(f'{x}, {y:.3f}' for x, y in zip(best_points, best_distances))
            print(f'\nEndzone: {best_points_repr}\n')

            for i, best_point in enumerate(best_points):
                self.positions.update(best_point.theta, best_point.fi)
                best_point, best_distance = self.estimate_spherical_positions()
                best_distance_points[best_distance] = best_point
                print(f'\nEndzone Iteration 1.{i + 1}: {best_point}, {best_distance:.3f}\n')

            best_point = best_distance_points[min(best_distance_points.keys())]
            self.positions.decrease()

            self.positions.update(best_point.theta, best_point.fi)
            best_point, best_distance = self.estimate_spherical_positions()
            self.positions.decrease()
            print(f'\nEndzone Iteration 2: {best_point}, {best_distance:.3f}')

        self.camera_initial.copy_from(self.best.camera)
        self.camera_previous.copy_from(self.best.camera)
        return self.best

    def estimate_spherical_positions(self, return_two_results=False):
        distances = {}

        for i in tqdm.tqdm(range(len(self.positions))):
            spherical_point = self.positions[i]
            cartesian_point = spherical_point.to_cartesian()
            c = self.game_center + cartesian_point
            best = BestResult()
            best.camera = Camera(c.x, c.y, c.z)
            self.look_at_center(best.camera)
            self.find_best_parameters(best, with_position=False)
            self.best.update_if_less(best.distance, best.camera, best.points, best.matches)
            distances[best.distance] = spherical_point

        distances = {k: v for k, v in sorted(distances.items(), key=lambda x: x[0])}
        best_distance = min(distances.keys())

        if not return_two_results:
            best_point = distances[next(iter(distances))]
            return best_point, best_distance
        else:
            best_points = []
            best_distances = []

            for i, (distance, spherical_point) in enumerate(distances.items()):
                if i == 0:
                    best_points.append(spherical_point)
                    best_distances.append(distance)
                else:
                    first_point = best_points[0]

                    if abs(first_point.fi - spherical_point.fi) > math.pi / 2.0:
                        best_points.append(spherical_point)
                        best_distances.append(distance)
                        break

            return best_points, best_distances

    def find_best_parameters(self, best, with_position=True, adjust_mapping=True):
        direction_params = {'position_shift': 0.0,
                            'rotation_shift': 0.0,
                            'shift_num': 1,
                            'multiply_coeff': 1.0,
                            'initial_index': 0}

        if not with_position:
            direction_params['rotation_shift'] = 1.0
            direction_params['multiply_coeff'] = 3.0

        else:
            if adjust_mapping:
                direction_params['position_shift'] = 0.1
                direction_params['rotation_shift'] = 0.1
                direction_params['multiply_coeff'] = 2.0
            else:
                direction_params['position_shift'] = 0.5
                direction_params['rotation_shift'] = 0.5
                direction_params['multiply_coeff'] = 3.0
                direction_params['initial_index'] = 1

        self.direction_list = DirectionList(direction_params)

        while True:
            self.direction_list.reset()

            best.index = -1
            index = self.__get_best_direction(best)

            if index == -1:
                break

            print_if(with_position, f'{best.distance:.3f}\t{best.camera.get_line_info(self.camera_previous)}: ', end='+')
            self.camera_previous.copy_from(best.camera)

            while True:
                is_success = self.__move_by_direction(best, best.camera, index)

                if is_success:
                    self.direction_list.increase()
                    print_if(with_position, end='+')
                else:
                    is_decreased = self.direction_list.decrease()

                    if not is_decreased:
                        print_if(with_position)
                        break

                    print_if(with_position, end='-')

        return best

    def get_mapping_result(self, best):
        info = best.camera.get_full_info(self.camera_initial)
        self.camera_initial.copy_from(best.camera)
        return info

    def look_at_center(self, camera):
        difference = camera.c - self.game_center
        camera.theta = difference.get_angle(self.rotation)

    def match_tracking_data(self, camera):
        projected_points = self.get_player_projections(camera).with_cluster(self.player_cluster)
        detected_points = self.detected_points.with_cluster(self.detection_cluster)
        distance, matches = projected_points.get_distance(detected_points)
        return projected_points, distance, matches

    def calculate_and_draw(self, camera):
        projected_points, distance, matches = self.match_tracking_data(camera)
        return self.draw_point_matches(projected_points, matches), distance

    def draw_point_matches(self, projected_points, matches):
        return draw_point_matches(self.frame_initial, projected_points, self.detected_points, matches)

    def get_player_projections(self, camera):
        projected_points = []

        for player in self.players:
            a_x, a_y, a_z = player.x, player.y, self.mean_player_height
            b_x, b_y = camera.get_2d_from_3d(a_x, a_y, a_z)
            point = Point(b_x * self.width, b_y * self.height, player.is_active)
            projected_points.append(point)

        return PointCloud(projected_points)

    def __get_best_direction(self, best):
        directed_camera = Camera()
        directed_camera.copy_from(best.camera)

        for index in range(len(self.direction_list)):
            self.__move_by_direction(best, directed_camera, index)

        return best.index

    def __move_by_direction(self, best, camera, index):
        directed_camera = Camera()
        directed_camera.copy_from(camera)

        c, theta = self.direction_list[index]
        directed_camera.c += c
        directed_camera.theta += theta

        projected_points, distance, matches = self.match_tracking_data(directed_camera)
        return best.update_if_less(distance, directed_camera, projected_points, matches, index=index)


In [ ]:
import cv2
import numpy as np

class TrackedPlayer:
    def __init__(self, name, data):
        self.name = name
        self.__tracking_data = data

    def __getitem__(self, frame_id):
        if isinstance(frame_id, int):
            return self.__tracking_data[frame_id]
        else:
            raise ValueError('Only an integer index is accepted.')

class Detection:
    def __init__(self, bbox):
        self.bbox = bbox
        self.point = bbox.lt.get_center(bbox.rb)
        self.player_name = ''

class TrackingSystem:
    increase_coeff = 1.5
    increase_min_factor = 5.0

    def __init__(self, video_name, players, bboxes):
        self.video_name = video_name
        self.players = []

        for i in range(len(players[next(iter(players))])):
            data = {}
            names = set()

            for frame_id in players.keys():
                data[frame_id] = [x for j, x in enumerate(players[frame_id]) if i == j][0]
                name = data[frame_id].name
                names.add(name)

            trackedPlayer = TrackedPlayer(name, data)
            self.players.append(trackedPlayer)

        self.detections = {}

        for frame_id in bboxes.keys():
            self.detections[frame_id] = [Detection(x) for x in bboxes[frame_id]]

        self.is_endzone = 'Endzone' in video_name

        self.snap_frame_id = get_ball_snap_frame_id(players)
        self.all_track_frame_ids = [x for x in players.keys() if x < len(self.detections)]
        self.used_track_frame_ids = []

        self.cameraMapping = None
        self.best = {}

        self.last_no_outlier_distance = None

    def get_camera_mapped(self, is_debug=True):
        frame, players, detected_points = self.get_data_by_frame_id(self.snap_frame_id)

        print(f'Players: {len(players)}')
        print(f'BBoxes: {len(detected_points)}\n')

        self.cameraMapping = CameraMapping(players, detected_points, frame, self.is_endzone)

        print('Find best camera rotation...\n')

        rotation = self.cameraMapping.get_best_rotations(is_debug=is_debug)

        print(f'\nRotation by X: {rotation.x}')
        print(f'Rotation by Y: {rotation.y}')
        print(f'Rotation by Z: {rotation.z}')

        print('\nFind best camera position...\n')

        best = self.cameraMapping.find_best_position()

        print(f'\n{self.cameraMapping.best.camera}')
        print(f'Best distance: {best.distance:.3f}\n')
        print('Adjust both position and rotation...\n')

        best = self.cameraMapping.find_best_parameters(best)
        projected_points, distance, matches = self.cameraMapping.match_tracking_data(best.camera)

        self.used_track_frame_ids.append(self.snap_frame_id)

        camera_info = self.cameraMapping.get_mapping_result(best)
        self.best[self.snap_frame_id] = best
        self.update_bbox_labels(self.snap_frame_id)
        self.last_no_outlier_distance = best.distance

        if is_debug:
            frame = self.cameraMapping.draw_point_matches(projected_points, matches)
            cv2.imwrite(f'../tracks/{self.snap_frame_id}.png', frame)

        print(f'\n{camera_info}')
        print(f'\nBest distance: {best.distance:.3f}\n')

    def track_by_sensors(self, is_debug=True):
        previous_frame_id = self.snap_frame_id
        previous_distance = self.best[previous_frame_id].distance

        for frame_id in self.all_track_frame_ids:
            if frame_id == self.snap_frame_id:
                continue

            frame, players, detected_points = self.get_data_by_frame_id(frame_id)
            self.cameraMapping.update_tracking_data(frame, players, detected_points)

            best = BestResult()
            best.copy_from(self.best[previous_frame_id])

            projected_points, distance, matches = self.cameraMapping.match_tracking_data(best.camera)
            print(f'{frame_id}: {distance:.3f}')

            is_distance_increased = False

            if self.is_distance_increased_sharply(distance, self.last_no_outlier_distance, matches):
                outlier_id = self.get_single_outlier(detected_points, distance, best.camera)

                if outlier_id is not None:
                    detected_points[outlier_id].is_active = False
                    projected_points, distance, matches = self.cameraMapping.match_tracking_data(best.camera)

                if self.is_distance_increased_sharply(distance, self.last_no_outlier_distance, matches):
                    if outlier_id is not None:
                        detected_points[outlier_id].is_active = True

                    self.cameraMapping.update_clusters(best.camera, distance, self.last_no_outlier_distance, self.increase_coeff)
                    projected_points, distance, matches = self.cameraMapping.match_tracking_data(best.camera)
                    print(f'\t{distance:.3f}')
                else:
                    print(f'\tDetection {outlier_id} is disabled.')
                    print(f'\t{distance:.3f}')

                is_distance_increased = True

            best.distance = 1e+6
            best = self.cameraMapping.find_best_parameters(best, adjust_mapping=False)
            print(f'\t{best.distance:.3f}')

            self.best[frame_id] = best
            self.update_bbox_labels(frame_id)

            if not is_distance_increased:
                self.last_no_outlier_distance = best.distance

            previous_frame_id = max(frame_id, self.snap_frame_id)
            previous_distance = self.best[previous_frame_id].distance

            self.used_track_frame_ids.append(frame_id)

            if is_debug:
                frame = self.cameraMapping.draw_point_matches(best.points, best.matches)
                cv2.imwrite(f'../tracks/{frame_id}.png', frame)

        camera_info = self.cameraMapping.get_mapping_result(best)
        print(f'\nFinal camera:\n\n{camera_info}')

    def track_by_iou(self, iou):
        frame_ids = list(sorted(self.detections.keys()))
        chunk_frame_ids = {frame_id: {'left': [], 'right': []} for frame_id in self.used_track_frame_ids}

        for frame_id in frame_ids:
            if frame_id in self.used_track_frame_ids:
                continue

            track_distance_dict = {x: abs(x - frame_id) for x in self.used_track_frame_ids}
            track_id = [k for k, v in sorted(track_distance_dict.items(), key=lambda x: x[1])][0]
            chunk_key = 'left' if frame_id < track_id else 'right'
            chunk_frame_ids[track_id][chunk_key].append(frame_id)

        for track_id, chunk_dict in chunk_frame_ids.items():
            for chunk_key, chunk_frame_ids in chunk_dict.items():
                previous_frame_id = track_id
                previous_bboxes = [x.bbox for x in self.detections[previous_frame_id]]
                sorted_frame_ids = reversed(chunk_frame_ids) if chunk_key == 'left' else chunk_frame_ids

                for frame_id in sorted_frame_ids:
                    now_bboxes = [x.bbox for x in self.detections[frame_id]]
                    match_dict = match_bboxes(now_bboxes, previous_bboxes, iou)

                    for first, second in match_dict.items():
                        player_name = self.detections[previous_frame_id][second].player_name
                        self.detections[frame_id][first].player_name = player_name

                    previous_bboxes = now_bboxes[:]
                    previous_frame_id = frame_id

    def get_results(self):
        rows = []

        for frame_id, detections in self.detections.items():
            detections_with_label = [x for x in detections if x.player_name != '']
            detections_with_label = sorted(detections_with_label, key=lambda x: x.player_name)

            for detection in detections_with_label:
                video_frame = f'{self.video_name}_{frame_id}'
                label = detection.player_name

                bbox = detection.bbox
                left, top = bbox.lt.x, bbox.lt.y
                width, height = bbox.get_width(), bbox.get_height()

                row = {'video_frame': video_frame, 'label': label,
                       'left': int(left), 'top': int(top),
                       'width': int(width), 'height': int(height)}

                rows.append(row)

        return rows

    def update_bbox_labels(self, frame_id):
        for match in self.best[frame_id].matches:
            player_name = self.players[match.first].name
            self.detections[frame_id][match.second].player_name = player_name

    def is_distance_increased_sharply(self, distance, previous, matches):
        if distance > previous * self.increase_coeff:
            distances = np.array([x.distance for x in matches])
            increase_factor = np.max(distances) / np.median(distances)

            if increase_factor > self.increase_min_factor:
                return True
            else:
                return False
        else:
            return False

    def get_single_outlier(self, detected_points, previous_distance, camera):
        best_distance = previous_distance
        best_outlier_id = None

        for detection_id in range(len(detected_points)):
            detected_points[detection_id].is_active = False
            _, distance, __ = self.cameraMapping.match_tracking_data(camera)

            if distance < best_distance:
                best_distance = distance
                best_outlier_id = detection_id

            detected_points[detection_id].is_active = True

        return best_outlier_id

    def get_data_by_frame_id(self, frame_id):
        players = [self.players[i][frame_id] for i in range(len(self.players))]
        detected_points = [x.point for x in self.detections[frame_id]]
        frame = read_specific_frame(self.video_name, frame_id)
        return frame, players, detected_points


In [ ]:
import os
from timeit import default_timer as timer

confidence = 0.5

submission_df_name = INPUT_PATH + 'sample_submission.csv'
submission_df = read_sample_submission(submission_df_name)

videonames = [x.split('.')[0] for x in os.listdir(INPUT_PATH + 'test/')]

test_df_name = INPUT_PATH + 'test_player_tracking.csv'
players = read_tracking_data(test_df_name, videonames)

baseline_df_name = INPUT_PATH + 'test_baseline_helmets.csv'
_, bboxes = read_baseline(baseline_df_name, videonames, confidence)

start = timer()

for videoname in videonames:
    players_now = players[videoname]
    bboxes_now = bboxes[videoname]

    print(f'Video name: {videoname}\n')
    trackingSystem = TrackingSystem(videoname, players_now, bboxes_now)

    trackingSystem.get_camera_mapped(is_debug=False)
    trackingSystem.track_by_sensors(is_debug=False)
    trackingSystem.track_by_iou(iou=0.35)
    results = trackingSystem.get_results()
    submission_df = submission_df.append(results, ignore_index=True)

stop = timer()
print(f'\nElapsed for: {stop - start:.3f} seconds.')
write_submission(OUTPUT_PATH + 'submission.csv', submission_df)